In [ ]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")

In [ ]:
@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 100000.0,
        dt    = 0.5,
        nₖₙₙ  = 50000,
        nₛₐᵥₑ = 1
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    )
)

In [ ]:
Par1, Par2 = (0.5,2.0,3.3), 0.05

FusionAGG = nothing
# @time @start_agg FusionAGG = FusionAggregate(
@time FusionAGG = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(Par1...), 
                ContractilePar(Par2)
            ),
            Float64.(readdlm("../../data/init/Sphere/9.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    Model
)

In [ ]:
points = FusionAGG.Position
idx    = FusionAGG.Simulation.Neighbor.idx
r_max  = FusionAGG.Simulation.Parameter.Force.rₘₐₓ

# Calculating Distance Matrix
threads=(8,8)
@cuda(
    threads=threads,
    blocks=cld.(size(points,1),threads),
    dist_kernel!(idx, points,r_max)
)

display(idx)

In [ ]:
idx_red = FusionAGG.Simulation.Neighbor.idx_red
idx_sum = FusionAGG.Simulation.Neighbor.idx_sum

# Reducing Distance Matrix to Nearest Neighbors
threads = 64
@cuda(
    threads=threads,
    blocks=cld.(size(points,1),threads),
    reduce_kernel!(idx,idx_red,idx_sum)
)

display(idx_red)
display(idx_sum)

In [ ]:
display(points)

In [ ]:
force = FusionAGG.Simulation.Force.F
ParForce = FusionAGG.Simulation.Parameter.Force
ParCont = FusionAGG.Simulation.Parameter.Contractile.fₚ
outline = FusionAGG.Geometry.outline

tₛᵢₘ  = Model.Time.tₛᵢₘ
nₛₐᵥₑ = Model.Time.nₛₐᵥₑ
nₖₙₙ  = Model.Time.nₖₙₙ
dt = Model.Time.dt

threads=(16,3)
@cuda(
    threads=threads,
    blocks=(cld.(size(points,1)+1,threads[1]),1),
    sum_force!(idx_red, points, force, ParForce, dt)
)
display(points)
display(force)

# Later

In [ ]:
# force = FusionAGG.Simulation.Force.F
# ParForce = FusionAGG.Simulation.Parameter.Force
# ParCont = FusionAGG.Simulation.Parameter.Contractile.fₚ
# outline = FusionAGG.Geometry.outline

# tₛᵢₘ  = 100.0
# nₛₐᵥₑ = Model.Time.nₛₐᵥₑ
# nₖₙₙ  = Model.Time.nₖₙₙ
# dt = Model.Time.dt

# # Calculating position of every cell on the fusion
# run(`rm Test.xyz `)
# using ProgressMeter
# @showprogress "Forces..." for step::Int=0:tₛᵢₘ/dt

#     # Saving data in a given time (nₛₐᵥₑ)
#     # if (step % Int(tₛᵢₘ/nₛₐᵥₑ)) == 1
#         open("Test.xyz", "a") do f
#             write(f, "$(size(points, 1))\n")
#             write(f, "t=$(step*dt)\n")
#             writedlm(f,hcat(outline,Matrix(points)), ' ')
#         end
#     # end

#     # # Calculating the kNN for the aggregate
#     # t_nₖₙₙ = step % nₖₙₙ+1
#     # if t_nₖₙₙ == 1
        
#         # # Calculating Distance Matrix
#         # threads=(8,8)
#         # @cuda(
#         #     threads=threads,
#         #     blocks=cld.(size(points,1),threads),
#         #     dist_kernel!(idx, points,r_max)
#         # )

#         # # Reducing Distance Matrix to Nearest Neighbors
#         # threads = 64
#         # @cuda(
#         #     threads=threads,
#         #     blocks=cld.(size(points,1),threads),
#         #     reduce_kernel!(idx,idx_red,idx_sum)
#         # )

#     # #     # Finding index contractile
#     # #     # Finding index contractile
#     # #     threads = (8,8)
#     # #     blocks  =cld.(size(points,1),threads)
#     # #     @cuda threads=threads blocks=blocks index_contractile!(idx_contractile,idx_sum,idx_red)
    
#     # end

#     # Compute the forces between each pair of particles in `agg` and their displacement.
#     threads=(16,3)
#     @cuda(
#         threads=threads,
#         blocks=(cld.(size(points,1)+1,threads[1]),1),
#         sum_force!(idx_red, points, force, ParForce, dt)
#     )
# end
# display(points)

In [ ]:
# idx_contractile = FusionAGG.Simulation.Neighbor.idx_cont

# # Finding index contractile
# threads = (8,8)
# @cuda(
#     threads=threads,
#     blocks=cld.(size(points,1),threads),
#     index_contractile!(idx_contractile,idx_sum,idx_red)
# )

# display(idx_contractile)